<a href="https://colab.research.google.com/github/kate-markina2709/Lab/blob/main/ML_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import numpy as np
import pandas as pd


In [ ]:
def extract_data_from_string(string):
    return string.split(";")


In [ ]:
def build_transition_matrix(data):
    matrix = np.full((66, 66), 1.0)

    for i in range(len(data) - 1):
        matrix[int(data[i]), int(data[i + 1])] += 1

    for i in range(66):
        matrix[i] = matrix[i] / matrix[i].sum()

    return matrix

In [ ]:
def calc_prob_window(seq, transition_matrix):
    prob = 1
    for i in range(len(seq) - 1):
        prob *= transition_matrix[int(seq[i]), int(seq[i + 1])]

    return prob

In [ ]:
def calc_threshhold(data, transition_matrix, window):
    prob_list = []

    if len(data) > window:
        for i in range(len(data) - window):
            window_data = data[i : i + window + 1]
            prob = calc_prob_window(window_data, transition_matrix)
            prob_list.append(prob)
    else:
        prob_list.append(calc_prob_window(data, transition_matrix))

    prob_array = np.array(prob_list)

    return prob_array.min()

In [ ]:
def check_on_anomalies(data, transition_matrix, thresh):
    prob = calc_prob_window(data, transition_matrix)
    return prob < thresh

In [ ]:
data = pd.read_csv("data.txt", sep=":", header=None, index_col=0, names=["User", "Data"])
data_true = pd.read_csv("data_true.txt", sep=":", header=None, index_col=0, names=["User", "Data"])
data_fake = pd.read_csv("data_fake.txt", sep=":", header=None, index_col=0, names=["User", "Data"])
print(data)
predictions_of_true = []
predictions_of_fake = []
window = 10

for i, row in data.iterrows():
    user_data = extract_data_from_string(row.Data)
    true_user_data = extract_data_from_string(data_true.Data[i])
    fake_user_data = extract_data_from_string(data_fake.Data[i])
    transition_matrix = build_transition_matrix(user_data)
    thresh = calc_threshhold(user_data, transition_matrix, window)

    predictions_of_true.append(
        check_on_anomalies(true_user_data, transition_matrix, thresh))

    predictions_of_fake.append(
        check_on_anomalies(fake_user_data, transition_matrix, thresh))

print(
    "Процент ложных срабатываний: ", sum(predictions_of_true) / len(predictions_of_true)
)
print(
    "Процент пропущенных выбросов: ",
    (len(predictions_of_fake) - sum(predictions_of_fake)) / len(predictions_of_fake),
)

                                                      Data
User                                                      
user1    41;41;41;41;41;41;41;41;41;41;41;41;41;41;41;4...
user2    55;55;55;55;55;28;28;28;28;28;56;56;56;56;28;2...
user3    41;41;41;41;41;41;41;41;41;41;41;41;41;41;41;4...
user4    15;15;15;15;15;15;15;15;15;15;15;15;15;15;15;1...
user5    35;35;35;35;35;35;35;35;35;35;35;35;35;35;35;3...
...                                                    ...
user196  45;43;43;45;45;43;45;43;45;45;45;43;43;45;45;4...
user197  43;45;43;45;45;43;45;43;45;43;45;45;45;45;45;4...
user198  45;43;43;45;45;45;45;43;45;43;43;45;45;45;45;4...
user199  0;0;0;0;0;0;0;0;0;0;0;3;3;3;0;0;3;0;0;0;0;0;3;...
user200  65;65;65;65;65;65;65;65;65;65;65;65;65;65;65;6...

[200 rows x 1 columns]
Процент ложных срабатываний:  0.01
Процент пропущенных выбросов:  0.225
